In [6]:
%load_ext autoreload
%autoreload 2

from IPython.display import display, HTML, Javascript
display(HTML('''<style>.jp-Cell.jp-CodeCell .jp-OutputArea:has(.jp-OutputArea-output) {background-color: #f9f9f9;border: 1px solid #ddd;padding: 10px;border-radius: 5px;margin: 5px 0;}</style><style>.jp-InputPrompt {display:none !important;}</style>'''))

# Building & exporting a `Case`
In this notebook, we will create a ROMS-MARBL [C-Star `Case`](https://c-star.readthedocs.io/en/latest/terminology.html#term-Case), by:
- Creating ROMS and MARBL [`BaseModel` objects](https://c-star.readthedocs.io/en/latest/terminology.html#term-BaseModel)
- Creating [`AdditionalCode` objects](https://c-star.readthedocs.io/en/latest/terminology.html#term-AdditionalCode) to represent namelist and additional source code files for ROMS
- Create [`InputDataset` ojects](https://c-star.readthedocs.io/en/latest/terminology.html#term-InputDataset) to tell C-Star where to find spatiotemporal data needed to run ROMS-MARBL
- Create a [`Dicretization` object](https://c-star.readthedocs.io/en/latest/terminology.html#term-Discretization) to tell C-Star how to partition the simulation (processor distribution and timestep)
- Bring these various objects together to make a [`ROMSComponent` object](https://c-star.readthedocs.io/en/latest/terminology.html#term-ROMSComponent) and a [`MARBLComponent` object](https://c-star.readthedocs.io/en/latest/terminology.html#term-MARBLComponent)
- Create a `Case` consisting of these two [`Component`s]((https://c-star.readthedocs.io/en/latest/terminology.html#term-Component)
- Export this `Case` to a [`blueprint` file](https://c-star.readthedocs.io/en/latest/terminology.html#term-blueprint) to be used in [the next notebook](https://c-star.readthedocs.io/en/latest/cstar_example_notebook.html)

In [2]:
import os
import cstar

Unloading the cpe module is insufficient to restore the system defaults.
Please run 'source
/opt/cray/pe/cpe/23.12/restore_lmod_system_defaults.[csh|sh]'. 
Running "module reset". Resetting modules to system default. The following $MODULEPATH directories have been removed: /opt/cray/pe/lmod/modulefiles/mpi/gnu/12.0/ofi/1.0/cray-mpich/8.0


Lmod is automatically replacing "gpu/1.0" with "cpu/1.0".


Note: the module "gpu" cannot be unloaded because it was not loaded.






## The structure of the Case:
[Here](https://c-star.readthedocs.io/en/latest/terminology.html#structure-of-c-star-case) you can get a general overview of a C-Star case. For our `roms_marbl_example`, the case structure breaks down like this:  
```
Case
├── MARBLComponent
│   └── base_model (MARBLBaseModel)
└── ROMSComponent
    ├── base_model (ROMSBaseModel)
    ├── namelists (AdditionalCode)
    ├── additional_source_code (AdditionalCode)
    ├── model_grid (ROMSInputDataset)
    ├── initial_conditions (ROMSInputDataset)
    ├── tidal_forcing (ROMSInputDataset)
    ├── surface_forcing (list of ROMSInputDatasets)
    ├── boundary_forcing (list of ROMSInputDatasets)
    └── discretization (ROMSDiscretization)

```
These are all the elements needed to create a unique, reproducible ROMS-MARBL simulation. You will notice that the `Component`, `BaseModel`, `InputDataset`, and `Discretization` objects here are specific to the object they describe (e.g. `ROMSBaseModel`). This is because there may be some unique attributes or operations associated with the `BaseModel` object describing ROMS that may be different to that describing MARBL, which has its own subclass `MARBLBaseModel`.

To build this case from the bottom up, we'll need to first build `BaseModel` objects for ROMS and MARBL

## Constructing the BaseModel objects
To initialize a [`BaseModel` object](https://c-star.readthedocs.io/en/latest/generated/cstar.base.BaseModel.html), we need:
- a `source_repo` (repository URL containing the base model source code) 
- a `checkout_target` (a point in the repository history we'd like to jump to). 

For ROMS we'll use the latest commit hash. For MARBL we'll use v0.45, around which the ROMS-MARBL driver was built:

In [2]:
from cstar.roms import ROMSBaseModel
from cstar.marbl import MARBLBaseModel

ModuleNotFoundError: No module named 'cstar'

In [1]:
roms_base_model = ROMSBaseModel(
    source_repo='https://github.com/CESR-lab/ucla-roms.git',
    checkout_target='main',
)

marbl_base_model = MARBLBaseModel(
    source_repo='https://github.com/marbl-ecosys/MARBL.git',
    checkout_target='marbl0.45.0',
)

NameError: name 'ROMSBaseModel' is not defined

In [4]:
print(roms_base_model)

ROMSBaseModel
-------------
source_repo : https://github.com/CESR-lab/ucla-roms.git (default)
checkout_target : 246c11fa537145ba5868f2256dfb4964aeb09a25
local_config_status: 0 (Environment variable ROMS_ROOT is present, points to the correct repository remote, and is checked out at the correct hash)


In [5]:
print(marbl_base_model)

MARBLBaseModel
--------------
source_repo : https://github.com/marbl-ecosys/MARBL.git (default)
checkout_target : marbl0.45.0 (corresponding to hash 6e6b2f7c32ac5427e6cf46de4222973b8bcaa3d9)
local_config_status: 0 (Environment variable MARBL_ROOT is present, points to the correct repository remote, and is checked out at the correct hash)


## Constructing the AdditionalCode objects

To construct an `AdditionalCode` object, we need a `location` pointing to a local or remote directory or repository.
In this example, we will use a remote repository, so we also supply a `subdir` (relative to the repository top level) and `checkout_target` argument to initialise the `AdditionalCode` instance.
We also need to provide a list of files pointing to our `AdditionalCode`.

In this example we are using MARBL and ROMS. As ROMS handles all input and output to MARBL, we only need `AdditionalCode` instances for ROMS: one for run-time files (namelists) and the other for compile-time files (such as ROMS' `.opt` files, which are used to set model parameters)

In [6]:
from cstar.base import AdditionalCode

In [7]:
roms_namelists = AdditionalCode(
    location = "https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git",
    subdir = "additional_code/ROMS/namelists",
    checkout_target = "a9762a46a36c09225423305a1aaa59bdeb984074",
    files = [
        "roms.in_TEMPLATE",
        "marbl_in",
        "marbl_tracer_output_list",
        "marbl_diagnostic_output_list"
    ]
)
print(roms_namelists)

AdditionalCode
--------------
Location: https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git
subdirectory: additional_code/ROMS/namelists
Working path: None
Exists locally: False (get with AdditionalCode.get())
Files:
    roms.in_TEMPLATE      (roms.in will be used by C-Star based on this template)
    marbl_in
    marbl_tracer_output_list
    marbl_diagnostic_output_list


In [8]:
roms_additional_source_code = AdditionalCode(
    location = "https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git",
    subdir = "additional_code/ROMS/source_mods",
    checkout_target = "a9762a46a36c09225423305a1aaa59bdeb984074",
    files = [
        "bgc.opt",
         "bulk_frc.opt",
         "cppdefs.opt",
         "diagnostics.opt",
         "ocean_vars.opt",
         "param.opt",
         "tracers.opt",
         "Makefile",
         "Make.depend",
    ]
)

print(roms_additional_source_code)

AdditionalCode
--------------
Location: https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git
subdirectory: additional_code/ROMS/source_mods
Working path: None
Exists locally: False (get with AdditionalCode.get())
Files:
    bgc.opt
    bulk_frc.opt
    cppdefs.opt
    diagnostics.opt
    ocean_vars.opt
    param.opt
    tracers.opt
    Makefile
    Make.depend


---

Note that for `roms_namelists`, in the first entry under `files`, the namelist file we begin with is a template. C-Star recognises the `_TEMPLATE` suffix and works with a local copy (in this case `roms.in`) that it will modify and use to run ROMS with user choices such as run length)

## Constructing the InputDataset objects
To run this case, we need different types of input dataset, each with a specialized subclass of the `InputDataset` class:
- a grid file supplying information about the domain to ROMS (`ROMSModelGrid`)
- An initial condition file from which to start the run (`ROMSInitialConditions`)
- boundary forcing files providing information at the edge of the domain (`ROMSBoundaryConditions`)
- surface forcing files providing information at the upper boundary (`ROMSSurfaceForcing`)
- tidal forcing files providing information on tidal constituents (`ROMSTidalForcing`)

To construct an `InputDataset` object, we need (see `cstar.InputDataset?`):
- a `location` : either a path or URL, pointing to this file.
- a `file_hash` : as our location is a URL in this example, we also need a [SHA256 checksum](https://en.wikipedia.org/wiki/Checksum) to verify any downloads

In our case, all the files associated with our `roms_marbl_example` case are small and [fit in a repository](https://github.com/CWorthy-ocean/input_datasets_roms_marbl_example). 

In [9]:
from cstar.roms import ROMSModelGrid, ROMSInitialConditions, ROMSTidalForcing, ROMSBoundaryForcing, ROMSSurfaceForcing

In [10]:
#Grid
roms_model_grid = ROMSModelGrid(
    location="https://github.com/CWorthy-ocean/input_datasets_roms_marbl_example/raw/main/roms_grd.nc",
    file_hash="fd537ef8159fabb18e38495ec8d44e2fa1b7fb615fcb1417dd4c0e1bb5f4e41d",
)
# Initial conditions
roms_initial_conditions = ROMSInitialConditions(
    location="https://github.com/CWorthy-ocean/input_datasets_roms_marbl_example/raw/main/MARBL_rst.20120103120000.nc",
    file_hash="fc3bbd039256edc89c898efda0eebc5c53773995598d59310bc6d57f454a6ddd",
)
# Tides
roms_tidal_forcing = ROMSTidalForcing(
    location="https://github.com/CWorthy-ocean/input_datasets_roms_marbl_example/raw/main/roms_tides.nc",
    file_hash="90db174ab174909f9bf27c13fa19995c03f680bcb80e7d012268505b48590338",
)
# Boundary
roms_phys_boundary_forcing = ROMSBoundaryForcing(
    location="https://github.com/CWorthy-ocean/input_datasets_roms_marbl_example/raw/main/roms_bry_2012.nc",
    file_hash="c3b0e14aae6dd5a0d54703fa04cf95960c1970e732c0a230427bf8b0fbbd8bf1",
)
roms_bgc_boundary_forcing = ROMSBoundaryForcing(
    location = "https://github.com/CWorthy-ocean/input_datasets_roms_marbl_example/raw/main/roms_bry_bgc_MARBL.nc",
    file_hash = "897a8df8ed45841a98b3906f2dd07750decc5c2b50095ba648a855c869c7d3ee",
)
# Surface
roms_bgc_surface_forcing = ROMSSurfaceForcing(
    location="https://github.com/CWorthy-ocean/input_datasets_roms_marbl_example/raw/main/roms_frc_bgc.nc",
    file_hash="621dd23691d87aa93c5cc582daf6c5f18333ed062ff934777d50b63346c3f84d",
)
roms_phys_surface_forcing = ROMSSurfaceForcing(
    location="https://github.com/CWorthy-ocean/input_datasets_roms_marbl_example/raw/main/roms_frc.201201.nc",
    file_hash="923049a9c2ab9ce77fa4a0211585e6848a12e87bf237e7aa310f693c3ac6abfa",
)


We can query each input dataset to get pertinent information about its state, e.g.:

In [11]:
print(roms_phys_boundary_forcing)

-------------------
ROMSBoundaryForcing
-------------------
Source location: https://github.com/CWorthy-ocean/input_datasets_roms_marbl_example/raw/main/roms_bry_2012.nc
file_hash: c3b0e14aae6dd5a0d54703fa04cf95960c1970e732c0a230427bf8b0fbbd8bf1
Working path: None ( does not yet exist. Call InputDataset.get() )


## Constructing the Discretization object
Lastly, we need to tell C-Star how we will be discretizing our components. MARBL piggybacks off the discretization of its host model, so we only need to create a `ROMSDiscretization` object. This contains:
- the time step (`time_step` , in seconds)
- the number of processors following x and y for running in parallel (`n_procs_x`, `n_procs_y`)

In [12]:
from cstar.roms import ROMSDiscretization

roms_discretization = ROMSDiscretization(time_step = 360,
                                         n_procs_x = 3,
                                         n_procs_y = 3)
print(roms_discretization)

ROMSDiscretization
------------------
time_step: 360s
n_procs_x: 3 (Number of x-direction processors)
n_procs_y: 3 (Number of y-direction processors)


## Putting it all together to build ROMS and MARBL components:
We now have everything we need to create the `MARBLComponent` and `ROMSComponent` objects that come together to make our Case.

In [13]:
from cstar.roms import ROMSComponent
from cstar.marbl import MARBLComponent

### MARBL:

In [14]:
marbl_component = MARBLComponent(
    base_model = marbl_base_model
)
print(marbl_component)

MARBLComponent
--------------
base_model: MARBLBaseModel instance (query using Component.base_model)


### ROMS
a ROMSComponent object (see `cstar.ROMSComponent?`) is a little more involved, not just because there are more objects going into it. Because we will be running this ROMS simulation in parallel, we should also supply information about how to distribute the domain among CPUs:

In [15]:
roms_component = ROMSComponent(
    base_model = roms_base_model,
    namelists = roms_namelists,
    additional_source_code = roms_additional_source_code,
    discretization = roms_discretization,
    model_grid = roms_model_grid,
    initial_conditions = roms_initial_conditions,
    tidal_forcing = roms_tidal_forcing,
    boundary_forcing = [roms_phys_boundary_forcing,roms_bgc_boundary_forcing],
    surface_forcing = [roms_phys_surface_forcing, roms_bgc_surface_forcing]
)
print(roms_component)

ROMSComponent
-------------
base_model: ROMSBaseModel instance (query using Component.base_model)
additional_source_code: AdditionalCode instance with 9 files (query using Component.additional_source_code)
namelists: AdditionalCode instance with 4 files (query using Component.namelists)
model_grid = <ROMSModelGrid instance>
initial_conditions = <ROMSInitialConditions instance>
tidal_forcing = <ROMSTidalForcing instance>
surface_forcing = <list of 2 ROMSSurfaceForcing instances>
boundary_forcing = <list of 2 ROMSBoundaryForcing instances>

Discretization:
ROMSDiscretization
------------------
time_step: 360s
n_procs_x: 3 (Number of x-direction processors)
n_procs_y: 3 (Number of y-direction processors)


## And finally, we can build the Case object:
This is instantiated using a list of components, a name string, and a "caseroot" - the local path where the case will be run. Additionally we choose a start date and end date for the run. The values below run the case for one model month, which may take several minutes to run (depending on your machine). **If you'd like to run the case more quickly**, modify `end_date` below:

In [16]:
from cstar import Case

In [17]:
roms_marbl_case = Case(
    components=[roms_component, marbl_component],
    name='roms_marbl_example_cstar_case',
    caseroot=os.getcwd() + '/roms_marbl_example_cstar_case',
    start_date='20120103 12:00:00',
    end_date='20120131 12:00:00',
)
print(roms_marbl_case)

C-Star Case
-----------
Name: roms_marbl_example_cstar_case
caseroot: /Users/dafyddstephenson/Code/my_c_star/examples/roms_marbl_example_cstar_case
start_date: 2012-01-03 12:00:00
end_date: 2012-01-31 12:00:00
Is setup: False
Valid date range:
valid_start_date: None
valid_end_date: None

It is built from the following Components (query using Case.components): 
   <ROMSComponent instance>
   <MARBLComponent instance>


/Users/dafyddstephenson/Code/my_c_star/cstar/case.py:103: RuntimeWarning: Valid start date not provided. Unable to check if simulation dates are out of range. Case objects should be initialized with valid_start_date and valid_end_date attributes.
  warnings.warn(
/Users/dafyddstephenson/Code/my_c_star/cstar/case.py:119: RuntimeWarning: Valid end date not provided. Unable to check if simulation dates are out of range. Case objects should be initialized with valid_start_date and valid_end_date attributes.
  warnings.warn(


## Visualizing the Case:
We can see how the caseroot directory will look once the case is set up using `Case.tree()`:

In [18]:
roms_marbl_case.tree()

/Users/dafyddstephenson/Code/my_c_star/examples/roms_marbl_example_cstar_case
├── input_datasets
│   └── ROMS
│       ├── roms_grd.nc
│       ├── MARBL_rst.20120103120000.nc
│       ├── roms_tides.nc
│       ├── roms_bry_2012.nc
│       ├── roms_bry_bgc_MARBL.nc
│       ├── roms_frc.201201.nc
│       └── roms_frc_bgc.nc
├── namelists
│   └── ROMS
│       ├── roms.in_TEMPLATE
│       ├── marbl_in
│       ├── marbl_tracer_output_list
│       └── marbl_diagnostic_output_list
└── additional_source_code
    └── ROMS
        ├── bgc.opt
        ├── bulk_frc.opt
        ├── cppdefs.opt
        ├── diagnostics.opt
        ├── ocean_vars.opt
        ├── param.opt
        ├── tracers.opt
        ├── Makefile
        └── Make.depend



## Setting the Case up
We can see from querying the `roms_marbl_case` object above (`Is setup: False`) that the case has not yet been set up locally. 

To set the case up, we run `roms_marbl_case.setup()`, which performs a series of actions:
- check the base model has been installed and, if not, ask the user if they would like to:
  - download ROMS and compile its dependencies
  - download and compile MARBL
- retrieve any additional code using `AdditionalCode.get()`
- retrieve any input datasets using `InputDataset.get()`

When the base models are installed, a file `cstar_local_config.py` will be created (or appended to) defining the user's environment such that these base models are available on your machine next time you import C-Star.

In [19]:
roms_marbl_case.setup()


Setting up ROMSComponent
-------------------------
Configuring ROMSComponent
-------------------------
ROMSBaseModel correctly configured. Nothing to be done

Fetching additional source code...
----------------------------------
Cloned repository https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git to /var/folders/1s/xjsjyqrj2rdft8014664qczc0000gn/T/tmpjqpchuel
Checked out a9762a46a36c09225423305a1aaa59bdeb984074 in git repository /var/folders/1s/xjsjyqrj2rdft8014664qczc0000gn/T/tmpjqpchuel
copying bgc.opt to /Users/dafyddstephenson/Code/my_c_star/examples/roms_marbl_example_cstar_case/additional_source_code/ROMS
copying bulk_frc.opt to /Users/dafyddstephenson/Code/my_c_star/examples/roms_marbl_example_cstar_case/additional_source_code/ROMS
copying cppdefs.opt to /Users/dafyddstephenson/Code/my_c_star/examples/roms_marbl_example_cstar_case/additional_source_code/ROMS
copying diagnostics.opt to /Users/dafyddstephenson/Code/my_c_star/examples/roms_marbl_example_cstar_

Cloned repository https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git to /var/folders/1s/xjsjyqrj2rdft8014664qczc0000gn/T/tmpu244_sq0
Checked out a9762a46a36c09225423305a1aaa59bdeb984074 in git repository /var/folders/1s/xjsjyqrj2rdft8014664qczc0000gn/T/tmpu244_sq0
copying roms.in_TEMPLATE to /Users/dafyddstephenson/Code/my_c_star/examples/roms_marbl_example_cstar_case/namelists/ROMS
copying template file /Users/dafyddstephenson/Code/my_c_star/examples/roms_marbl_example_cstar_case/namelists/ROMS/roms.in_TEMPLATE to editable version /Users/dafyddstephenson/Code/my_c_star/examples/roms_marbl_example_cstar_case/namelists/ROMS/roms.in
copying marbl_in to /Users/dafyddstephenson/Code/my_c_star/examples/roms_marbl_example_cstar_case/namelists/ROMS
copying marbl_tracer_output_list to /Users/dafyddstephenson/Code/my_c_star/examples/roms_marbl_example_cstar_case/namelists/ROMS
copying marbl_diagnostic_output_list to /Users/dafyddstephenson/Code/my_c_star/examples/roms_marbl


Setting up MARBLComponent
--------------------------
Configuring MARBLComponent
--------------------------
MARBLBaseModel correctly configured. Nothing to be done


## Saving the Case to a YAML file
We can save all the information associated with this case to a YAML file using `Case.persist(filename)`
If we want to run this exact Case again, we can use the `Case.from_blueprint(filename)` method:

In [20]:
roms_marbl_case.persist(roms_marbl_case.caseroot / 'roms_marbl_case.yaml')

In [21]:
print((roms_marbl_case.caseroot/"roms_marbl_case.yaml").read_text())

registry_attrs:
  name: roms_marbl_example_cstar_case
components:
- component:
    component_type: ROMS
    base_model:
      source_repo: https://github.com/CESR-lab/ucla-roms.git
      checkout_target: 246c11fa537145ba5868f2256dfb4964aeb09a25
    additional_source_code:
      location: https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git
      subdir: additional_code/ROMS/source_mods
      checkout_target: a9762a46a36c09225423305a1aaa59bdeb984074
      files:
      - bgc.opt
      - bulk_frc.opt
      - cppdefs.opt
      - diagnostics.opt
      - ocean_vars.opt
      - param.opt
      - tracers.opt
      - Makefile
      - Make.depend
    namelists:
      location: https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git
      subdir: additional_code/ROMS/namelists
      checkout_target: a9762a46a36c09225423305a1aaa59bdeb984074
      files:
      - roms.in_TEMPLATE
      - marbl_in
      - marbl_tracer_output_list
      - marbl_diagnostic_output_list